#**Step-by-Step Guide to Switch to Google Gemini**


#**Step 1: Get Your Google Gemini API Key**
    
  Go to Google AI Studio: https://aistudio.google.com/

Sign in with your Google account.

Click on "Get API key" in the top left.

Click "Create API key in new project".

Copy the generated API key. Keep it safe!

#**Step 2: Update Project Dependencies (requirements.txt)**

You'll need the Google AI Python library.

Add the new library to your requirements.txt file. You can remove openai.


# requirements.txt


1.   google-generativeai
2.   python-dotenv
3.   requests
4.   tweepy
5.   pandas
6.   pandas-ta
7.   coinbase-advanced-py


In [1]:
# .env file

# Coinbase API Keys
COINBASE_API_KEY="YOUR_COINBASE_API_KEY"
COINBASE_API_SECRET="YOUR_COINBASE_API_SECRET"

# Twitter (X) API Keys
TWITTER_BEARER_TOKEN="YOUR_TWITTER_BEARER_TOKEN"

# Google Gemini API Key (NEW)
GOOGLE_API_KEY="YOUR_NEW_GEMINI_API_KEY"

**Load the new key in config.py:**

In [ ]:
# config.py
import os
from dotenv import load_dotenv

load_dotenv()

COINBASE_API_KEY = os.getenv("COINBASE_API_KEY")
COINBASE_API_SECRET = os.getenv("COINBASE_API_SECRET")
TWITTER_BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY") # Add this line

**Step 4: Modify the Bot Logic (bot.py)**

This is the main change. We will update the bot.py file to use the Gemini client and API calls.
Replace the OpenAI parts with the new Gemini code. Here is the updated bot.py file:

In [ ]:
# bot.py
import time
import json
import logging
import google.generativeai as genai # <-- CHANGED
import config
from api_clients import get_product_data, get_latest_tweet, execute_order
from strategy import get_technical_signals, get_elon_signal

# --- Configure the Gemini client --- (CHANGED)
genai.configure(api_key=config.GOOGLE_API_KEY)

class TradingBot:
    def __init__(self, product_id="DOGE-USD", trade_size_quote=10.0):
        self.product_id = product_id
        self.trade_size_quote = trade_size_quote
        self.in_position = False
        self.bought_at = 0.0

        # --- Initialize the Gemini Model --- (NEW)
        self.ai_model = genai.GenerativeModel('gemini-1.5-flash')

    def run(self):
        """The main trading loop."""
        logging.info("Starting AI Trading Bot (powered by Google Gemini)...")
        while True:
            try:
                self.make_decision()
                logging.info("Waiting for next cycle (5 minutes)...")
                time.sleep(300)
            except KeyboardInterrupt:
                logging.info("Bot shutting down.")
                break
            except Exception as e:
                logging.error(f"An error occurred in the main loop: {e}")
                time.sleep(60)

    def make_decision(self):
        """Gathers data, analyzes it, and decides whether to trade."""
        market_data = get_product_data(self.product_id)
        if market_data is None: return
        tech_signals = get_technical_signals(market_data)

        latest_tweet = get_latest_tweet("elonmusk")
        elon_signal = get_elon_signal(latest_tweet)

        final_decision, reason = self.ask_ai(tech_signals, latest_tweet, elon_signal)
        logging.info(f"Gemini Decision: {final_decision}. Reason: {reason}")

        if "BUY" in final_decision and not self.in_position:
            order = execute_order('BUY', self.product_id, self.trade_size_quote)
            if order:
                self.in_position = True
                self.bought_at = tech_signals.get('price', 0)
        elif "SELL" in final_decision and self.in_position:
            current_price = tech_signals.get('price', 0)
            if self.bought_at > 0:
                base_size_to_sell = self.trade_size_quote / self.bought_at
                order = execute_order('SELL', self.product_id, f"{base_size_to_sell:.6f}")
                if order:
                    profit = (current_price - self.bought_at) * base_size_to_sell
                    logging.info(f"Position sold for an estimated profit of ${profit:.2f}")
                    self.in_position = False
                    self.bought_at = 0.0
            else:
                logging.warning("Sell decision made, but 'bought_at' price is zero. Cannot calculate size.")


    # --- THIS IS THE UPDATED FUNCTION ---
    def ask_ai(self, tech_signals, tweet_text, elon_signal):
        """Asks Google Gemini for a final trading decision based on structured data."""

        # Override logic for strong signals
        if elon_signal == "STRONG_BUY":
            return "BUY", "Elon Musk tweet detected as a strong catalyst."

        prompt = f"""
        As a crypto trading analyst bot, your goal is to make a final decision (BUY, SELL, HOLD)
        based on technical indicators and social signals. Prioritize caution; a 'HOLD' is better than a risky trade.
        Your response MUST be in a valid JSON format: {{"decision": "YOUR_DECISION", "reason": "Your brief explanation."}}

        ---
        Data for {self.product_id}:

        Current Position Status: I am currently {'IN a position' if self.in_position else 'NOT in a position'}.

        Technical Analysis:
        - Current Price: {tech_signals.get('price')}
        - RSI(14): {tech_signals.get('rsi')} (Under 35 is often oversold/buy, over 65 is overbought/sell)
        - Technical Strategy Suggestion: '{tech_signals.get('decision')}'.

        Social Signal:
        - Latest Tweet from elonmusk: "{tweet_text}"

        Based on all this information, provide your final decision in the required JSON format.
        """

        try:
            # Configure Gemini to output JSON
            generation_config = genai.types.GenerationConfig(
                response_mime_type="application/json"
            )
            response = self.ai_model.generate_content(
                prompt,
                generation_config=generation_config
            )

            # The response text will be a JSON string
            data = json.loads(response.text)
            return data.get("decision", "HOLD"), data.get("reason", "No reason provided.")

        except Exception as e:
            logging.error(f"Error getting response from Gemini: {e}")
            return "HOLD", "Error communicating with the AI model."